In [1]:
import heapq 
import os
class BinaryTreeNode:
    
    def __init__(self, value, frequency):
        self.value = value
        self.frequency = frequency
        self.left = None
        self.right = None
        
    def __lt__(self, other):
        return self.frequency < other.frequency
    
    def __eq__(self, other):
        return self.frequency == other.frequency

class HuffmanCoding:
    
    
    def __init__(self, path):
        self.path = path
        self.__heap = []
        self.__codes = {}
        self.__reverseCodes = {}
        
    def __make_frequency_dict(self, text):
        d = {}
        for char in text:
            if char not in d:
                d[char] = 0
            d[char] = d[char] + 1
            
        return d
    
    def __buildTree(self):
        while (len(self.__heap) > 1):
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            
            freqSum = binary_tree_node_1.frequency + binary_tree_node_2.frequency
            
            NewNode = BinaryTreeNode(None,freqSum)
            NewNode.left = binary_tree_node_1
            NewNode.right = binary_tree_node_2
            heapq.heappush(self.__heap,NewNode)
            
        return
    
    def __buildCodesHelper(self, root, curr_bits):
        if root is None:
            return 
        
        if root.value is not None:
            self.__codes[root.value] = curr_bits
            self.__reverseCodes[curr_bits] = root.value
            return
        
        self.__buildCodesHelper(root.left, curr_bits + "0")
        self.__buildCodesHelper(root.right, curr_bits + "1")    
        
    def __buildCodes(self):
        root = heapq.heappop(self.__heap)
        self.__buildCodesHelper(root, "")
        
    
    def __buildHeap(self, d):
        for key in d:
            frequency = d[key]
            Binary_Tree_Node = BinaryTreeNode(key, frequency)
            heapq.heappush(self.__heap, Binary_Tree_Node)
    
    def __getEncodedText(self, text):
        encodedText = " "
        for char in text:
            encodedText = encodedText + self.__codes[char]
        return encodedText
    
    def __getPaddedEncodedText(self, encoded_text):
        padded_amount = 8 - (len(encoded_text)%8)
        for i in range(padded_amount):
            encoded_text = encoded_text + '0'
            
        padded_info = "{0:08b}".format(padded_amount)
        
        padded_encoded_text = padded_info + encoded_text
        return padded_encoded_text
    
    def __getBytesArray(self, paddedEncodedText):
        array = []
        for i in range(0,len(paddedEncodedText),8):
            byte = paddedEncodedText[i:i+8]
            array.append(int(byte,2))
            
        return array
    
    def compress(self):
        file_name, file_extension = os.path.splitext(self.path)
        output_path = file_name + ".bin"
        
        with open(self.path, 'r+') as file, open(output_path, 'wb') as output:

            text = file.read()
            text = text.rstrip()
            
            d = self.__make_frequency_dict(text)
            
            self.__buildHeap(d)
            
            self.__buildTree()
            
            self.__buildCodes()
            
            EncodedText = self.__getEncodedText(text)
            
            padded_encoded_text = self.__getPaddedEncodedText(EncodedText)
            
            bytesArray = self.__getBytesArray(padded_encoded_text)
            
            final_bytes = bytes(bytesArray)
            
            output.write(final_bytes)
        print("compressed")
        return output_path
    
    def __removePadding(self, text):
        padded_info = text[0:8:1]
        extra_padding = int(padded_info, 2)
        
        text_after_padding_removed = text[8:-1*extra_padding]
        
        return text_after_padding_removed
    
    def __decodeText(self, text):
        
        decoded_text = ""
        current_bits = ""
        
        for bits in text:
            current_bits = current_bits + bits
            if current_bits in self.__reverseCodes:
                character = self.__reverseCodes[current_bits]
                decoded_text = decoded_text + character
                current_bits = ""
                
        return decoded_text
    
    def decompress(self, input_path):
        file_name, file_extension = os.path.splitext(self.path)
        output_path = file_name + "_decompressed" + ".txt"
        
        with open(input_path, 'rb') as file, open(output_path, 'w') as output:
            bit_string = ""
            byte = file.read(1)
            while byte:
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8,'0')
                bit_string = bit_string + bits
                byte = file.read(1)
            
            actual_text = self.__removePadding(bit_string)
            decompressed_text = self.__decodeText(actual_text)
            output.write(decompressed_text)
            
        return

In [2]:
path = "/Users/yashlakhwani/CODING/PYTHON/DSA/HUFFMAN CODING/sample.txt"
h = HuffmanCoding(path)
output_path = h.compress()
h.decompress(output_path)

compressed
